In [1]:
import re
import pandas as pd
import datetime
import mysql.connector


In [11]:
conexao = mysql.connector.connect(host='localhost',user='root',password='',database='producao')
cursor = conexao.cursor()

In [12]:
# antes de rodar voce deve criar manualmente um novo registro na tabela rpis_lidas com a revista e data que irá ler
# sempre que alterar manualmente qualquer tabel pelo phmyadmin não de rodar de novo a linha de connect acima
# copie o arquivo TXT e XML na pasta revistas
def LerFilename(revista):
    comando = f'select * from rpis_lidas where rpi="{revista}"'
    cursor.execute(comando)
    resultado = cursor.fetchall()
    #print(len(resultado))
    for row in resultado:
        #print(row[0])
        data_revista = row[1]
    df = pd.DataFrame(resultado)
    data = df[1].astype("string")
    str_data_revista = str(data_revista)
    d=datetime.datetime.strptime(str_data_revista, "%Y-%m-%d")
    #print(d.year)
    aux = str(d.strftime("%d%m%Y"))
    filename = f'revistas/Patente_{revista}_{aux}.xml'
    return filename, str_data_revista

revista = 2711 # 2022 tem revistas de 2661 a 2712
filename, data_revista = LerFilename(revista)
print(filename)
print(data_revista)

revistas/Patente_2711_20122022.xml
2022-12-20


In [13]:
def ConverteNumero(x):
    s = str(x)
    s = s.replace("BR","")
    s = s.replace(" ","")
    pos = s.find('-')
    return s[0:pos]

In [14]:
def ConverteData(data):
    d=datetime.datetime.strptime(data, "%d/%m/%Y")
    return f'{d.year:04}-{d.month:02}-{d.day:02}'

In [15]:
def LerRPI(filename,data_revista):
    file = open(file=filename,mode='r',encoding="utf8")
    file_content_raw=file.read()
    text1=re.compile("<despacho>")
    file_content=text1.split(file_content_raw)
    file.close()
    texto=file_content[0]
    t = re.compile(r'\d{2}/../\d{4}')
    check = t.findall(texto)
    data = ''.join(check)
    data=ConverteData(data)
    if data_revista == data:
        print(filename) 
    else:
        print(filename+"\n Data da RPI "+data_revista+" não confere com a data gravada dentro do XML "+data) 
    return file_content

file_content = LerRPI(filename,data_revista)
print("Total registros: "+str(len(file_content)-1))

revistas/Patente_2711_20122022.xml
Total registros: 4040


In [16]:
def identificado_ca(numero):
    if numero[0:1].isnumeric():
        i = 14
        if numero.find("-")==-1:
            i = 12;
        if len(numero)==i and numero[0:2]=='13':
            return True
        else:
            return False
    else:
        if numero[0:1]=='C':
            return True;
        else:
            return False;
        
def identificado_mu(numero):
    if numero[0:1].isnumeric():
        i = 14
        if numero.find("-")==-1:
            i = 12;
        if len(numero)==i and (numero[0:2]=='20' or numero[0:2]=='21' or numero[0:2]=='22'):
            return True
        else:
            return False
    else:
        if numero[0:2]=='MU':
            return True;
        else:
            return False;
        
def identificado_pi(numero):
    if numero[0:1].isnumeric():
        i = 14
        if numero.find("-")==-1:
            i = 12;
        if len(numero)==i and (numero[0:2]=='10' or numero[0:2]=='11' or numero[0:2]=='12'):
            return True
        else:
            return False
    else:
        if numero[0:2]=='PI':
            return True;
        else:
            return False;

#https://www.gov.br/inpi/pt-br/backup/legislacao-arquivo/docs/resolucao_74-2013-deposito_dos_pedidos_de_patentes.pdf
#https://wipolex-res.wipo.int/edocs/lexdocs/laws/pt/br/br106pt.html
def gera_digito(numero): 
    print("iniciando")
    numero=numero.strip()
    num = len(numero)
    total = 0
    print(numero)

    if len(numero)==12: # ZZXXXXYYYYYY
        multiplicador = 2
        while num>0:
            total = total + int(numero[num-1:num])*multiplicador
            num = num - 1
            multiplicador = multiplicador + 1
            if multiplicador>9: 
                multiplicador = 2

        resto = 11 - (total%11)
        if resto==10 or resto==11:
            resto=0
    else:
        multiplicador = 2
        while num>=3:
            total = total + int(numero[num-1:num])*multiplicador
            num = num - 1
            multiplicador = multiplicador + 1

        resto = 11 - (total%11)
        if resto==10 or resto==11:
            resto=0
    return resto

def montar_numerocd(numero): # monta numero sem digito verificador
    numero = numero.upper().strip()
    if numero[0:1].isnumeric(): 
        numero = numero[0:12]
    else:
        numero = numero[0:9]
    digito = str(gera_digito(numero))
    numerocd = numero + "-" + digito
    return numerocd

def montar_numerosd(numero): # monat numero sem digito verificador
    numero = numero.strip()
    if numero[0:2]=='BR':
        numerosd = numero[2:12].upper().strip()
    elif numero[0:1].isnumeric(): 
        numerosd = numero[0:12].upper().strip()
    else:
        numerosd = numero[0:9].upper().strip()
    return numerosd


def InserePiMuPi(numero,comentario,data):
    comentario = comentario.upper().strip()
    comentario = comentario.replace(",", "")
    comentario = comentario.replace('    ',' ')
    comentario = comentario.replace('   ',' ')
    comentario = comentario.replace('  ',' ')
    comentario = comentario.replace('É','E')
    comentario = comentario.replace('"','')
    comentario = comentario.replace('PI-','PI')
    comentario = comentario.replace('PI -','PI')
    comentario = comentario.replace('PI:','PI')
    comentario = comentario.replace('PP-','PP')
    comentario = comentario.replace('PP -','PP')
    comentario = comentario.replace('PP:','PP')
    comentario = comentario.replace('MU-','MU')
    comentario = comentario.replace('MU -','MU')
    comentario = comentario.replace('MU:','MU')
    comentario = comentario.replace('BR:','BR')
    comentario = comentario.replace('BR-','BR')
    comentario = comentario.replace('BR -','BR')

    str = [ "PARA PATENTE DE INVENÇÂO SOB O Nº",
            "ALTERADA A NUMERAÇÃO E A NATUREZA PARA :",
            "MUDADA A NATUREZA PARA",
            "PARA MODELO DE NATUREZA SOB O NÚMERO",
            "PARA MODELO DE UTILIDADE E A NUMERAÇÃO PARA",
            "PARA MODELO DE UTILIDADE SOB NÚMERO:",
            "PARA MODELO DE UTILIDADE SOB NÚMERO",
            "PARA MODELO DE UTILIDADE SOB Nº",
            "PARA MODELO DE UTILIDADE (",
            "PARA MODELO DE UTILIDADE COM A NUMERAÇÃO",
            "PARA MU (",
            "PARA MODELO DE UTILIDADE",
            "COM O NOVO NÚMERO",
            "E A NUMERAÇÃO PARA",
            "NÚMERO DO PEDIDO PARA O",
            "PARA PATENTE DE INVENÇÃO SOB O NÚMERO",
            "PARA PATENTE DE INVENÇÃO SOB NÚMERO",
            "PARA PATENTE DE INVENÇÃO SOB NUMERO",
            "PARA PATENTE DE INVENÇÃO SOB O NUMERO",
            "PARA PATENTE DE INVENÇÃO COM A NUMERAÇÃO",
            "PARA:",
            "PARA"]
    
    for j in str:
        pos = comentario.find(j)
        if pos>=0:
            break
    pos = pos + len(j)
    numero2b = comentario[pos::1].strip() # string[start:end:step]
    numero2b = numero2b.replace(' ','')
    numero2b = numero2b.replace('BR','')
    numero2bsd = montar_numerosd(numero2b)
    
    numero1b = numero.replace(' ','')
    numero1b = numero1b.replace('BR','')
    numero1bsd = montar_numerosd(numero1b)
    print(comentario)
    comando = f"select * from pimupi where numero1='{numero1bsd}' or numero2='{numero2bsd}'"
    cursor.execute(comando)
    if cursor.fetchone() == None:
            comando = f"insert into pimupi (numero1, numero2, data) values ('{numero1bsd}','{numero2bsd}','{data}')"
            try:
                cursor.execute(comando)
                conexao.commit()
                print(comando)
            except mysql.connector.Error as err:
                if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                    print("Conferir user name or password")
                elif err.errno == errorcode.ER_BAD_DB_ERROR:
                    print("Banco de dados inativo")
                else:
                    print(err)
                    print("Erro "+str(numero)+" na gravação na tabela pimupi")
                    
    #numero1bcd = montar_numerocd('BR102022015433')
    #print(numero1bcd)
    #numero1bcd = montar_numerocd('PI8700456')
    #print(numero1bcd)
    #if identificado_pi('102022015433'):
    #    print("1 - identificado invenção")
    #if identificado_pi('PI9802334-2'):
    #    print("2 - identificado invenção")
    #if identificado_pi('202022015433'):
    #    print("3 - identificado invenção")
    #if identificado_pi('MU9802334-2'):
    #    print("4 - identificado invenção")
    #if identificado_mu('202022015433'):
    #    print("5 - identificado modelo utilidade")
    #if identificado_mu('MU9802334-2'):
    #    print("6 - identificado modelo utilidade")
    #if identificado_ca('MU9802334-2'):
    #    print("7 - identificado certificado adição")
    #if identificado_ca('C19802334-2'):
    #    print("8 - identificado certificado adição")
    return()

In [17]:
codigo_tag = re.compile('<codigo>(.*?)</codigo>')
titulo_tag = re.compile('<titulo>(.*?)</titulo>')
comentario_tag = re.compile("<comentario inid=\"co\">(.*?)</comentario>") 
nome_completo_tag = re.compile("<nome-completo>(.*?)</nome-completo>") 
data_deposito_tag = re.compile("<data-deposito inid=\"22\">(.*?)</data-deposito>") 

numero_tag = re.compile("<numero inid=\"21\">(.*?)</numero>") 
numero_a2_tag = re.compile("<numero inid=\"21\" kindcode=\"A2\">(.*?)</numero>") 
numero_e2_tag = re.compile("<numero inid=\"21\" kindcode=\"E2\">(.*?)</numero>") 
numero_u2_tag = re.compile("<numero inid=\"21\" kindcode=\"U2\">(.*?)</numero>") 
numero_a8_tag = re.compile("<numero inid=\"21\" kindcode=\"A8\">(.*?)</numero>") 
numero_u8_tag = re.compile("<numero inid=\"21\" kindcode=\"U8\">(.*?)</numero>") 
numero_21b1_tag = re.compile("<numero inid=\"21\" kindcode=\"B1\">(.*?)</numero>") 
numero_21b8_tag = re.compile("<numero inid=\"21\" kindcode=\"B8\">(.*?)</numero>") 
numero_21y1_tag = re.compile("<numero inid=\"21\" kindcode=\"Y1\">(.*?)</numero>") 
numero_21y8_tag = re.compile("<numero inid=\"21\" kindcode=\"Y8\">(.*?)</numero>") 
numero_21e8_tag = re.compile("<numero inid=\"21\" kindcode=\"E8\">(.*?)</numero>") 

numero_11a8_tag = re.compile("<numero inid=\"11\" kindcode=\"A8\">(.*?)</numero>") 
numero_b1_tag = re.compile("<numero inid=\"11\" kindcode=\"B1\">(.*?)</numero>") 
numero_b8_tag = re.compile("<numero inid=\"11\" kindcode=\"B8\">(.*?)</numero>") 
numero_y1_tag = re.compile("<numero inid=\"11\" kindcode=\"Y1\">(.*?)</numero>") 
numero_y8_tag = re.compile("<numero inid=\"11\" kindcode=\"Y8\">(.*?)</numero>") 
numero_c8_tag = re.compile("<numero inid=\"11\" kindcode=\"C8\">(.*?)</numero>") 
numero_f1_tag = re.compile("<numero inid=\"11\" kindcode=\"F1\">(.*?)</numero>") 
numero_f8_tag = re.compile("<numero inid=\"11\" kindcode=\"F8\">(.*?)</numero>") 
numero_g8_tag = re.compile("<numero inid=\"11\" kindcode=\"G8\">(.*?)</numero>") 

data_fase_nacional_tag = re.compile("<data-fase-nacional inid=\"85\">(.*?)</data-fase-nacional>") 
titulo_pedido_tag = re.compile("<titulo inid=\"54\">(.*?)</titulo>") 
titular_lista_tag = re.compile("<titular-lista>[\s\S]*</titular-lista>")

codigo_list, titulo_list, comentario_list, nome_completo_list, data_deposito_list, numero_list, uf_list, sigla_list, numero_pct_list, data_pct_list, publicacao_internacional_list, numero_ompi_list, data_ompi_list, classe_internacional_list, classe_nacional_list, sigla_prioridade_list, numero_prioridade_list, data_prioridade_list, prioridade_list, prioridade_interna_list, data_fase_nacional_list, titulo_pedido_list, inventor_nome_completo_list, divisao_pedido_list, data_divisao_pedido_list, data_rpi_list, data_concessao_list = ([] for i in range(27))

for line in file_content:
    
    codigo=codigo_tag.findall(line)
    titulo=titulo_tag.findall(line)
    comentario=comentario_tag.findall(line)
    comentario = ', '.join(comentario)
    
       
    #to find inventors
    data_deposito=None
    data_fase_nacional=None
    numero=None
    nome_completo=None
    inventor_nome_completo=None
    titulo_pedido=""
    uf=None
    sigla=None
    numero_pct=" "
    data_pct=" "
    numero_ompi=" "
    data_ompi=" "
    classe_internacional=None
    str_classe = ''
    classe_nacional=None
    str_classe_nacional = ''
    sigla_prioridade=" "
    numero_prioridade=" "
    data_prioridade=" "
    str_prioridade=''
    numero_prioridade_interna=" "
    data_prioridade_interna=" "
    str_prioridade_interna=''
    numero_divisao_pedido=" "
    data_divisao_pedido=" "
    data_rpi=" "
    data_concessao=" "
    
    processos=re.findall("<processo-patente>[\s\S]*</processo-patente>",line)
    for processo in processos:
        data_deposito=data_deposito_tag.findall(processo)
        data_deposito=', '.join(data_deposito) # converte lista em string
        numero=numero_tag.findall(processo)
        if len(numero)==0:
            numero=numero_a2_tag.findall(processo)
            if len(numero)==0:
                numero=numero_u2_tag.findall(processo)
                if len(numero)==0:
                    numero=numero_a8_tag.findall(processo)
                    if len(numero)==0:
                        numero=numero_u8_tag.findall(processo)
                        if len(numero)==0:
                            numero=numero_b1_tag.findall(processo)
                            if len(numero)==0:
                                numero=numero_b8_tag.findall(processo)
                                if len(numero)==0:
                                    numero=numero_y1_tag.findall(processo)
                                    if len(numero)==0:
                                        numero=numero_y8_tag.findall(processo)
                                        if len(numero)==0:
                                            numero=numero_e2_tag.findall(processo)
                                            if len(numero)==0:
                                                numero=numero_21b1_tag.findall(processo)
                                                if len(numero)==0:
                                                    numero=numero_21b8_tag.findall(processo)
                                                    if len(numero)==0:
                                                        numero=numero_21y1_tag.findall(processo)
                                                        if len(numero)==0:
                                                            numero=numero_21y8_tag.findall(processo)
                                                            if len(numero)==0:
                                                                numero=numero_c8_tag.findall(processo)
                                                                if len(numero)==0:
                                                                    numero=numero_f1_tag.findall(processo)
                                                                    if len(numero)==0:
                                                                        numero=numero_11a8_tag.findall(processo)
                                                                        if len(numero)==0:
                                                                            numero=numero_21e8_tag.findall(processo)
                                                                            if len(numero)==0:
                                                                                numero=numero_f8_tag.findall(processo)
                                                                                if len(numero)==0:
                                                                                    numero=numero_g8_tag.findall(processo)
        numero = ', '.join(numero)
        data_fase_nacional=data_fase_nacional_tag.findall(processo)
        data_fase_nacional = ', '.join(data_fase_nacional)
        titulo_pedido=titulo_pedido_tag.findall(processo)
        titulo_pedido = ', '.join(titulo_pedido)

        s1 = '15.10'
        s2 = '15.12'
        if s1 in codigo or s2 in codigo:
            InserePiMuPi(numero,comentario,data_revista)
        
        #concessao (16.3)
        concessoes_lista=re.findall("<concessao inid=\"45\">[\s\S]*</concessao>",processo)
        for concessao_lista in concessoes_lista:
            data_concessao=re.findall("<data>(.*?)</data>",concessao_lista)
            
        #publicacao_nacional (16.1)
        publicacoes_nacional_lista=re.findall("<publicacao-nacional inid=\"43\">[\s\S]*</publicacao-nacional>",processo)
        for publicacao_nacional_lista in publicacoes_nacional_lista:
            data_rpi=re.findall("<data-rpi>(.*?)</data-rpi>",publicacao_nacional_lista)
            
        #divisao_pedido
        divisao_pedidos_lista=re.findall("<divisao-pedido inid=\"62\">[\s\S]*</divisao-pedido>",processo)
        for divisao_pedido_lista in divisao_pedidos_lista:
            data_divisao_pedido=re.findall("<data-deposito>(.*?)</data-deposito>",divisao_pedido_lista)
            numero_divisao_pedido=re.findall("<numero>(.*?)</numero>",divisao_pedido_lista)
            
        #titular_lista
        titulares_lista=re.findall("<titular-lista>[\s\S]*</titular-lista>",processo)
        for titular_lista in titulares_lista:
            titulares=re.findall("<titular sequencia=\"1\" inid=\"71\">[\s\S]*</titular>",titular_lista)
            for titular in titulares:
                nome_completo=nome_completo_tag.findall(titular)
                nome_completo = '; '.join(nome_completo)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                for endereco in enderecos:
                    uf=re.findall("<uf>(.*?)</uf>",endereco)
                    uf = '; '.join(uf)
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                        sigla = '; '.join(sigla)
                        
            titulares=re.findall("<titular sequencia=\"1\" inid=\"73\">[\s\S]*</titular>",titular_lista)
            for titular in titulares:
                nome_completo=nome_completo_tag.findall(titular)
                nome_completo = '; '.join(nome_completo)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                for endereco in enderecos:
                    uf=re.findall("<uf>(.*?)</uf>",endereco)
                    uf = '; '.join(uf)
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                        sigla = '; '.join(sigla)

        #inventor_lista
        inventores_lista=re.findall("<inventor-lista>[\s\S]*</inventor-lista>",processo)
        for inventor_lista in inventores_lista:
            #print(inventor_lista)
            inventores=re.findall("<inventor sequencia=\"1\" inid=\"72\">[\s\S]*</inventor>",inventor_lista)
            for inventor in inventores:
                inventor_nome_completo=nome_completo_tag.findall(inventor)
                inventor_nome_completo = '; '.join(inventor_nome_completo)
                        
        #pedido_internacional
        pedidos_internacional_lista=re.findall("<pedido-internacional inid=\"86\">[\s\S]*</pedido-internacional>",processo)
        for pedido_internacional_lista in pedidos_internacional_lista:
            #print(pedido_internacional_lista)
            numeros_pct=re.findall("<numero-pct>[\s\S]*</numero-pct>",pedido_internacional_lista)
            for numero_pct_lista in numeros_pct:
                numero_pct=re.findall("<numero-pct>(.*?)</numero-pct>",numero_pct_lista)
            datas_pct=re.findall("<data-pct>[\s\S]*</data-pct>",pedido_internacional_lista)
            for data_pct_lista in datas_pct:
                data_pct=re.findall("<data-pct>(.*?)</data-pct>",data_pct_lista)

        #publicacao_internacional
        publicacoes_internacional_lista=re.findall("<publicacao-internacional inid=\"87\">[\s\S]*</publicacao-internacional>",processo)
        for publicacao_internacional_lista in publicacoes_internacional_lista:
            numeros_ompi=re.findall("<numero-ompi>[\s\S]*</numero-ompi>",publicacao_internacional_lista)
            for numero_ompi_lista in numeros_ompi:
                numero_ompi=re.findall("<numero-ompi>(.*?)</numero-ompi>",numero_ompi_lista)
            datas_ompi=re.findall("<data-ompi>[\s\S]*</data-ompi>",publicacao_internacional_lista)
            for data_ompi_lista in datas_ompi:
                data_ompi=re.findall("<data-ompi>(.*?)</data-ompi>",data_ompi_lista)

        #classificacao_internacional
        classificacoes_internacional_lista=re.findall("<classificacao-internacional-lista>[\s\S]*</classificacao-internacional-lista>",processo)
        for classificacao_internacional_lista in classificacoes_internacional_lista:
            for i in range(1,20):
                #print(f"<classificacao-internacional inid=\"51\" sequencia=\"{i}\" ano=\"2006.01\" >[\s\S]*</classificacao-internacional>")
                classificacoes_internacional=re.findall(f"<classificacao-internacional inid=\"51\" sequencia=\"{i}\" ano=\"2006.01\">[\s\S]*</classificacao-internacional>",classificacao_internacional_lista)
                for classificacao_internacional in classificacoes_internacional:
                    classe_internacional=re.findall(f"<classificacao-internacional inid=\"51\" sequencia=\"{i}\" ano=\"2006.01\">(.*?)</classificacao-internacional>",classificacao_internacional)
                    if i==1:
                        str_classe = ', '.join(classe_internacional) # converte lista em string
                    else:
                        str_classe = str_classe+'; '+', '.join(classe_internacional) # converte lista em string
                    #print(', '.join(numero)+' '+str_classe)

        #classificacao_nacional
        classificacoes_nacional_lista=re.findall("<classificacao-nacional-lista>[\s\S]*</classificacao-nacional-lista>",processo)
        for classificacao_nacional_lista in classificacoes_nacional_lista:
            for i in range(1,20):
                #print(f"<classificacao-nacional inid=\"52\" sequencia=\"{i}\" >[\s\S]*</classificacao-nacional>")
                classificacoes_nacional=re.findall(f"<classificacao-nacional inid=\"52\" sequencia=\"{i}\">[\s\S]*</classificacao-nacional>",classificacao_nacional_lista)
                for classificacao_nacional in classificacoes_nacional:
                    classe_nacional=re.findall(f"<classificacao-nacional inid=\"52\" sequencia=\"{i}\">(.*?)</classificacao-nacional>",classificacao_nacional)
                    if i==1:
                        str_classe_nacional = ', '.join(classe_nacional) # converte lista em string
                    else:
                        str_classe_nacional = str_classe_nacional+'; '+', '.join(classe_nacional) # converte lista em string
                    #print(', '.join(numero)+' '+str_classe)
                    
        #prioridade_unionista
        prioridades_unionista_lista=re.findall("<prioridade-unionista-lista>[\s\S]*</prioridade-unionista-lista>",processo)
        for prioridade_unionista_lista in prioridades_unionista_lista:
            for i in range(1,5):
                prioridades_unionista=re.findall(f"<prioridade-unionista sequencia=\"{i}\" inid=\"30\">[\s\S]*</prioridade-unionista>",prioridade_unionista_lista)
                for prioridade_unionista in prioridades_unionista:
                    #print(prioridade_unionista)
                    sigla_prioridade=re.findall("<sigla-pais inid=\"33\">(.*?)</sigla-pais>",prioridade_unionista)
                    numero_prioridade=re.findall("<numero-prioridade inid=\"32\">(.*?)</numero-prioridade>",prioridade_unionista)
                    data_prioridade=re.findall("<data-prioridade inid=\"31\">(.*?)</data-prioridade>",prioridade_unionista)
                    sigla_prioridade_str=sigla_prioridade[0]
                    numero_prioridade_str=numero_prioridade[0]
                    data_prioridade_str=data_prioridade[0]
                    if i==1:
                        str_prioridade = sigla_prioridade_str+numero_prioridade_str+' '+data_prioridade_str # converte lista em string
                    else:
                        str_prioridade = str_prioridade+'; '+sigla_prioridade_str+numero_prioridade_str+' '+data_prioridade_str

                    numero_str = ', '.join(numero)
                    if numero_str=='BR 11 2022 020312-8':
                        print(f"<prioridade-unionista sequencia=\"{i}\" inid=\"30\">[\s\S]*</prioridade-unionista>")
                        print(sigla_prioridade)
                        print(numero_prioridade)
                        print(data_prioridade)
                        print(str_prioridade)
 
        #prioridade_interna
        prioridades_interna_lista=re.findall("<prioridade-interna-lista>[\s\S]*</prioridade-interna-lista>",processo)
        for prioridade_interna_lista in prioridades_interna_lista:
            for i in range(1,5):
                prioridades_interna=re.findall(f"<prioridade-interna sequencia=\"{i}\" inid=\"66\">[\s\S]*</prioridade-interna>",prioridade_interna_lista)
                for prioridade_interna in prioridades_interna:
                    numero_prioridade_interna=re.findall("<numero-prioridade>(.*?)</numero-prioridade>",prioridade_interna)
                    data_prioridade_interna=re.findall("<data-prioridade>(.*?)</data-prioridade>",prioridade_interna)
                    numero_prioridade_interna_str=numero_prioridade_interna[0]
                    data_prioridade_interna_str=data_prioridade_interna[0]
                    if i==1:
                        str_prioridade_interna = numero_prioridade_interna_str+' '+data_prioridade_interna_str # converte lista em string
                    else:
                        str_prioridade_interna = str_prioridade_interna+'; '+numero_prioridade_interna_str+' '+data_prioridade_interna_str

                
    # checking length of gid is not equal to 0 then do append to all the lists
    if len(codigo)!=0:                             
        codigo_list.append(codigo[0])
        titulo_list.append(titulo[0])
        comentario_list.append(comentario)
        data_deposito_list.append(data_deposito)
        numero_list.append(numero)
        data_fase_nacional_list.append(data_fase_nacional)
        titulo_pedido_list.append(titulo_pedido)
        nome_completo_list.append(nome_completo)
        inventor_nome_completo_list.append(inventor_nome_completo)
        uf_list.append(uf)
        sigla_list.append(sigla)
        numero_pct_list.append(numero_pct[0])
        data_pct_list.append(data_pct[0])
        numero_ompi_list.append(numero_ompi[0])
        data_ompi_list.append(data_ompi[0])
        classe_internacional_list.append(str_classe)
        classe_nacional_list.append(str_classe_nacional)
        prioridade_list.append(str_prioridade)
        sigla_prioridade_list.append(sigla_prioridade[0])
        numero_prioridade_list.append(numero_prioridade[0])
        data_prioridade_list.append(data_prioridade[0])
        prioridade_interna_list.append(str_prioridade_interna)
        divisao_pedido_list.append(numero_divisao_pedido[0])
        data_divisao_pedido_list.append(data_divisao_pedido[0])
        data_rpi_list.append(data_rpi[0])
        data_concessao_list.append(data_concessao[0])
        


PEDIDO RENUMERADO DEVIDO A MUDANÇA DE NATUREZA DE BR102016016457-5 PARA BR202016016457-0.
insert into pimupi (numero1, numero2, data) values ('102016016457','202016016457','2022-12-20')
PEDIDO RENUMERADO DEVIDO A MUDANÇA DE NATUREZA DE BR102017010814-7 PARA BR202017010814-2.
insert into pimupi (numero1, numero2, data) values ('102017010814','202017010814','2022-12-20')
PEDIDO RENUMERADO DEVIDO A MUDANÇA DE NATUREZA DE BR102018002069-2 PARA BR202018002069-8.
insert into pimupi (numero1, numero2, data) values ('102018002069','202018002069','2022-12-20')
PEDIDO RENUMERADO DEVIDO A MUDANÇA DE NATUREZA DE BR132018008879-2 PARA BR202018008879-9.
insert into pimupi (numero1, numero2, data) values ('132018008879','202018008879','2022-12-20')
PEDIDO RENUMERADO DEVIDO A MUDANÇA DE NATUREZA DE BR102018014992-0 PARA BR202018014992-5.
insert into pimupi (numero1, numero2, data) values ('102018014992','202018014992','2022-12-20')
PEDIDO RENUMERADO DEVIDO A MUDANÇA DE NATUREZA DE BR112015026213-9 PAR

In [18]:
data_frame = pd.DataFrame(
    {'despacho': codigo_list,
     'numero': numero_list,
     'data_deposito': data_deposito_list,
     'data_fase_nacional': data_fase_nacional_list,
     'data_publicacao_nacional': data_rpi_list,
     'data_concessao': data_concessao_list,
     'numero_divisao': divisao_pedido_list,
     'data_divisao': data_divisao_pedido_list,
     'titulo': titulo_pedido_list,
     'depositante': nome_completo_list,
     'uf': uf_list,
     'sigla': sigla_list,
     'inventor': inventor_nome_completo_list,
     'numero_pct': numero_pct_list,
     'data_pct': data_pct_list,
     'numero_ompi': numero_ompi_list,
     'data_ompi': data_ompi_list,
     'classe_internacional': classe_internacional_list,
     'classe_nacional': classe_nacional_list,
     'prioridade': prioridade_list,
     'sigla_prioridade': sigla_prioridade_list,
     'numero_prioridade': numero_prioridade_list,
     'data_prioridade': data_prioridade_list,
     'prioridade_interna': prioridade_interna_list,
     'comentario': comentario_list,
     'texto_despacho': titulo_list
    })

In [19]:
data_frame["numero_new"] = data_frame["numero"].apply(ConverteNumero)
# https://pt.stackoverflow.com/questions/564421/aplicar-fun%c3%a7%c3%a3o-lamba-para-uma-subtra%c3%a7%c3%a3o-e-soma-python/564425#564425

In [20]:
print(len(data_frame.index))
data_frame[["despacho","numero_new","data_deposito","data_fase_nacional"]]


4040


,despacho,numero_new,data_deposito,data_fase_nacional
0,135,112018072304,24/07/2017,
1,1.1,112021019356,,
2,1.1,112021019357,,
3,1.1,112021019870,,
4,1.1,112022001992,,
...,...,...,...,...
4035,28.30,102021024740,07/12/2021,
4036,28.30,112019017753,03/04/2018,
4037,28.30,112021008063,30/10/2019,
4038,28.30,112021022815,13/05/2020,


In [21]:
for i, infos in data_frame.iterrows():
    numero = infos.numero_new
    despacho = infos.despacho
    comando = f"INSERT INTO arquivadosxml (id, despacho, numero, data, divisao, anulado, prmexame) VALUES (NULL, '{despacho}', '{numero}', '{data_revista}', '', 0, 0);"
    cursor.execute(comando)
    conexao.commit()

In [22]:
#confere total de registros gravados
comando = f'select count(*) as total from arquivadosxml where data="{data_revista}"'
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    if len(data_frame.index) == row[0]:
        print("Sucesso na gravação do total de "+str(row[0])+" registros !")
    else:
        print("Total de registros gravados: "+str(row[0])+" registros, quando toral era de "+len(data_frame.index))



Sucesso na gravação do total de 4040 registros !


In [23]:
#confere total de registros gravados com a tabela padrão arquivados
comando = f'select * from arquivados where data="{data_revista}"'
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    despacho = row[1]
    numero = row[2]
    data = row[3]
    df = data_frame.loc[data_frame['numero_new']==numero]
    if df.empty:
        print("Não encontrei "+str(numero)+" na revista lida")
    else:
        max = len(df)
        ok = False
        for i in range(max):
            id = df.index[i]
            #print("Encontrei "+str(numero)+" na revista lida")
            despacho_rpi=df.loc[id].despacho
            if despacho_rpi in ['100','102','103','104','106','111','112','113','115','116','120','121','130','131','132','133','134','135','136','137','138','139','140','141']:
                despacho_rpi = 'PR - Recursos'
            if despacho_rpi in ['200','201','204','205','206','210','211','212','213','214','215','216','217','218','219','220']:
                despacho_rpi = 'PR - Nulidades'
            if despacho_rpi == despacho:
                ok = True
                
        if (ok == False):
            print("Despacho diferente "+str(numero)+" "+str(despacho)+" "+str(despacho_rpi))
        #print(numero,despacho,despacho_rpi)
    #break
print("Processamento encerrado")

Processamento encerrado


In [24]:
#confere total de registros gravados com a tabela padrão arquivados
comando = f'select * from arquivadosxml where data="{data_revista}"'
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    despacho = row[1]
    numero = row[2]
    data = row[3]
    if despacho in ['100','102','103','104','106','111','112','113','115','116','120','121','130','131','132','133','134','135','136','137','138','139','140','141']:
        despacho = 'PR - Recursos'
    if despacho in ['200','201','204','205','206','210','211','212','213','214','215','216','217','218','219','220']:
        despacho = 'PR - Nulidades'
    comando = f'select * from arquivados where numero="{numero}" and despacho="{despacho}" and data="{data}"'
    cursor.execute(comando)
    if cursor.fetchone() == None:
        print("Não encontrei "+str(numero)+", "+str(despacho)+" na tabela arquivados")

print("Processamento encerrado")

Processamento encerrado


In [40]:
def AtualizaPrmExame():
    #atualiza campo prmexame que registra a etapa de exame técnico
    etapa = 1
    total = 0
    for ano in range(2022,2023,1):
        comando = f"select * from arquivados where despacho in ('6.1','7.1','9.1','9.2') and anulado=0 and year(data)={ano} order by numero,data asc"
        cursor.execute(comando)
        resultado = cursor.fetchall()
        for row in resultado:
            id = row[0]
            despacho = row[1]
            numero = row[2]
            data = row[3]
            prmexame = row[6]

            numero1 = numero
            numero2 = numero
            comando = f"select * from pimupi where numero1='{numero}' or numero2='{numero}'"
            cursor.execute(comando)
            resultado2 = cursor.fetchall()
            for row in resultado2:
                numero1 = row[0]
                numero2 = row[1]

            etapa =1
            comando = f"select * from arquivados where (numero='{numero1}' or numero='{numero2}') and despacho in ('6.1','7.1','9.1','9.2','11.2') and anulado=0 order by data asc"
            cursor.execute(comando)
            resultado2 = cursor.fetchall()
            for row in resultado2:
                id = row[0]
                despacho = row[1]
                numero = row[2]
                data = row[3]
                prmexame = row[6]

                if prmexame!=etapa:
                    comando = f"update arquivados set prmexame={etapa} where id={id} and numero='{numero}'"
                    #cursor.execute(comando)
                    #if cursor.fetchone() == None:
                    #    print("Não encontrei "+str(numero)+", "+str(despacho)+" na tabela arquivados")
                    print(comando+";")

                etapa = etapa + 1
                
            total = total + 1
            if (total==200): break

    print("Processamento encerrado")
    return

AtualizaPrmExame()

update arquivados set prmexame=1 where id=4259777 and numero='102012000340';
update arquivados set prmexame=2 where id=4368129 and numero='102012000340';
update arquivados set prmexame=3 where id=4453619 and numero='102012000340';
update arquivados set prmexame=1 where id=4530425 and numero='102012000368';
update arquivados set prmexame=1 where id=4565735 and numero='102012000753';
update arquivados set prmexame=1 where id=4318537 and numero='102012001293';
update arquivados set prmexame=2 where id=4438369 and numero='102012001293';
update arquivados set prmexame=1 where id=4356344 and numero='102012001773';
update arquivados set prmexame=2 where id=4442662 and numero='102012001773';
update arquivados set prmexame=1 where id=4323116 and numero='102012001875';
update arquivados set prmexame=2 where id=4450579 and numero='102012001875';
update arquivados set prmexame=1 where id=4200634 and numero='102012001876';
update arquivados set prmexame=2 where id=4318262 and numero='102012001876';

update arquivados set prmexame=1 where id=4334308 and numero='102012026665';
update arquivados set prmexame=2 where id=4473407 and numero='102012026665';
update arquivados set prmexame=3 where id=4548659 and numero='102012026665';
update arquivados set prmexame=1 where id=4334308 and numero='102012026665';
update arquivados set prmexame=2 where id=4473407 and numero='102012026665';
update arquivados set prmexame=3 where id=4548659 and numero='102012026665';
update arquivados set prmexame=1 where id=3879860 and numero='102012026789';
update arquivados set prmexame=2 where id=4620335 and numero='102012026789';
update arquivados set prmexame=1 where id=4424215 and numero='102012026954';
update arquivados set prmexame=2 where id=4474179 and numero='102012026954';
update arquivados set prmexame=1 where id=4424215 and numero='102012026954';
update arquivados set prmexame=2 where id=4474179 and numero='102012026954';
update arquivados set prmexame=1 where id=4410314 and numero='102012027245';

In [25]:
def ConverteData(s1):
    s1 = str(s1) #converte em string
    s1 = s1.lstrip() #elimina espaços em branco à esquerda
    s1 = s1.rstrip() #elimina espaços em branco à direita 03/12/2020
    if s1!="" and s1 is not None:
        s1 = s1[6:10]+"-"+s1[3:5]+"-"+s1[0:2]
    else:
        s1 = None
    return s1

def InsereArqPatentes():
    #insere registro novo na tabela arqpatentes a partir da tabela arquivadosxml
    comando = f"select * from arquivadosxml where data='{data_revista}' and despacho in ('1.1','1.3','2.4','3.1','3.2','3.6') and anulado=0"
    total = 0
    cursor.execute(comando)
    resultado = cursor.fetchall()
    for row in resultado:
        id = row[0]
        despacho = row[1]
        numero = row[2]
        data = row[3]
        #numero='112015032339' #1.3. data_nacional <> data_deposito
        #despacho='1.3'
        #numero='102021009009' #3.1 data_nacional = data_deposito
        #despacho='3.1'
        #numero='112022023076' #1.1 data_nacional = null
        #despacho='1.1'

        numero1 = numero
        numero2 = numero
        comando = f"select * from pimupi where numero1='{numero}' or numero2='{numero}'"
        cursor.execute(comando)
        resultado2 = cursor.fetchall()
        for row in resultado2:
            numero1 = row[0]
            numero2 = row[1]

        comando = f"select * from arqpatentes where numero='{numero1}' or numero='{numero2}'"
        cursor.execute(comando)
        if cursor.fetchone() == None:
            df1 = data_frame.loc[data_frame['numero_new']==numero]
            #print(df1)
            id = df1.index[0]
            #print(numero+" "+despacho)

            data_deposito = None
            s1 = df1.loc[id].data_deposito
            data_deposito = ConverteData(s1)
            if data_deposito is not None:
                data_deposito = ConverteData(s1)
            else:
                s1 = df1.loc[id].data_pct
                data_deposito = ConverteData(s1)
            #print(data_deposito)

            data_nacional = None
            s1 = df1.loc[id].data_fase_nacional
            data_nacional = ConverteData(s1)
            if data_nacional is None:
                data_nacional = data_deposito
            if despacho=='1.1':
                data_nacional = None
            #print(data_nacional)

            #break
            if data_deposito is None and data_nacional is None:
                comando = f"INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('{numero}', null, null, null, null, null, null, null, '')"
            elif data_deposito is None:
                comando = f"INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('{numero}', null, '{data_nacional}', null, null, null, null, null, '')"
            elif data_nacional is None:
                comando = f"INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('{numero}', '{data_deposito}', null, null, null, null, null, null, '')"
            else:
                comando = f"INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('{numero}', '{data_deposito}', '{data_nacional}', null, null, null, null, null, '')"

            try:
                cursor.execute(comando)
                conexao.commit()
                print(comando+";")
                total = total + 1
            except mysql.connector.Error as err:
                if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                    print("Conferir user name or password")
                elif err.errno == errorcode.ER_BAD_DB_ERROR:
                    print("Banco de dados inativo")
                else:
                    print(err)
                    print("Erro "+str(numero)+" na gravação na tabela arqpatentes")
                
            #if (total==2000): break

    print("Processamento encerrado")
    return

InsereArqPatentes()

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112021019356', '2021-05-13', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112021019357', '2021-05-13', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112021019870', '2021-07-14', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022001992', '2021-05-17', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022020155', '2021-04-05', null, null,

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025003', '2021-06-11', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025004', '2021-06-08', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025005', '2021-06-09', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025007', '2021-06-09', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025008', '2021-06-08', null, null,

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025093', '2021-06-18', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025094', '2021-06-07', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025095', '2021-06-11', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025099', '2021-06-10', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025100', '2020-06-12', null, null,

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025170', '2021-06-09', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025171', '2021-06-10', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025172', '2021-06-14', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025173', '2021-06-23', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025174', '2020-12-22', null, null,

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025227', '2021-06-10', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025228', '2021-06-01', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025229', '2021-06-11', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025230', '2021-06-08', null, null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022025231', '2020-06-11', null, null,

...

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022023621', '2021-05-20', '2022-11-21', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022023627', '2021-04-15', '2022-11-21', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022023636', '2021-05-14', '2022-11-21', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022023637', '2021-05-25', '2022-11-21', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('11202202

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022023832', '2021-05-27', '2022-11-23', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022023835', '2021-05-25', '2022-11-23', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022023836', '2021-02-26', '2022-11-23', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022023838', '2021-06-02', '2022-11-23', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('11202202

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022024090', '2021-05-20', '2022-11-25', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022024091', '2021-06-01', '2022-11-25', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022024095', '2021-05-21', '2022-11-25', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('112022024096', '2021-05-18', '2022-11-25', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('11202202

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('102021011086', '2021-06-08', '2021-06-08', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('102021011088', '2021-06-08', '2021-06-08', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('102021011092', '2021-06-09', '2021-06-09', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('102021011096', '2021-06-09', '2021-06-09', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('10202101

INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('102021020932', '2021-10-19', '2021-10-19', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('102021024917', '2021-12-09', '2021-12-09', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('102022004392', '2022-03-10', '2022-03-10', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('102022007393', '2022-04-18', '2022-04-18', null, null, null, null, null, '');
INSERT INTO arqpatentes (numero, data_deposito, data_nacional, pedexame, concessao, anuidade, extincao, dataout, despacho_out) VALUES ('10202201

In [26]:
def AtualizaConcessao():
    print("Atualiza campo concessao na tabela arqpatentes")
    comando = f"select * from arqpatentes where concessao is null"
    total = 0
    cursor.execute(comando)
    resultado = cursor.fetchall()
    for row in resultado:
        numero = row[0]

        numero1 = numero
        numero2 = numero
        comando = f"select * from pimupi where numero1='{numero}' or numero2='{numero}'"
        cursor.execute(comando)
        resultado2 = cursor.fetchall()
        for row in resultado2:
            numero1 = row[0]
            numero2 = row[1]

        try:
            comando = f"select * from arquivadosxml where (numero='{numero1}' or numero='{numero2}') and despacho='16.1' and anulado=0"
            cursor.execute(comando)
            resultado2 = cursor.fetchall()
            if resultado2 != None:
                for row in resultado2:
                    data = row[3]
                    comando = f"update arqpatentes set concessao='{data}' where numero='{numero}'"
                    cursor.execute(comando)
                    conexao.commit()
                    print(comando+";")
                    total = total + 1
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Conferir user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Banco de dados inativo")
            else:
                print(err)
                print("Erro "+str(numero)+" na gravação na tabela arqpatentes")
                
        #if (total==2000): break

    print("Atualiza campo concessao na tabela arqpatentes verificando as concessões anuladas")
    comando = f"select * from arqpatentes where concessao is not null"
    total = 0
    cursor.execute(comando)
    resultado = cursor.fetchall()
    for row in resultado:
        numero = row[0]
        data = row[4]

        numero1 = numero
        numero2 = numero
        comando = f"select * from pimupi where numero1='{numero}' or numero2='{numero}'"
        cursor.execute(comando)
        resultado2 = cursor.fetchall()
        for row in resultado2:
            numero1 = row[0]
            numero2 = row[1]

        try:
            comando = f"select * from arquivadosxml where (numero='{numero1}' or numero='{numero2}') and data='{data}' and despacho='16.1' and anulado<>0"
            cursor.execute(comando)
            resultado2 = cursor.fetchone()
            if resultado2 != None:
                comando = f"update arqpatentes set concessao=null where numero='{numero}'"
                #cursor.execute(comando)
                #conexao.commit()
                print(comando+";")
                total = total + 1
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Conferir user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Banco de dados inativo")
            else:
                print(err)
                print("Erro "+str(numero)+" na gravação na tabela arqpatentes")
                
        #if (total==2000): break

    print("Processamento encerrado")
    return

AtualizaConcessao()

Atualiza campo concessao na tabela arqpatentes
update arqpatentes set concessao='2022-12-20' where numero='102014015027';
update arqpatentes set concessao='2022-12-20' where numero='102014021463';
update arqpatentes set concessao='2022-12-20' where numero='112013021460';
update arqpatentes set concessao='2022-12-20' where numero='112014012539';
update arqpatentes set concessao='2022-12-20' where numero='112014012892';
update arqpatentes set concessao='2022-12-20' where numero='112014020602';
update arqpatentes set concessao='2022-12-20' where numero='112014027348';
update arqpatentes set concessao='2022-12-20' where numero='112015000449';
update arqpatentes set concessao='2022-12-20' where numero='112015013387';
update arqpatentes set concessao='2022-12-20' where numero='112015014590';
update arqpatentes set concessao='2022-12-20' where numero='112015017299';
update arqpatentes set concessao='2022-12-20' where numero='112015019347';
update arqpatentes set concessao='2022-12-20' where n

In [27]:
def AtualizaExtincao():
    print("Atualiza campo extincao na tabela arqpatentes")
    comando = f"select * from arqpatentes where extincao is null"
    total = 0
    cursor.execute(comando)
    resultado = cursor.fetchall()
    for row in resultado:
        numero = row[0]

        numero1 = numero
        numero2 = numero
        comando = f"select * from pimupi where numero1='{numero}' or numero2='{numero}'"
        cursor.execute(comando)
        resultado2 = cursor.fetchall()
        for row in resultado2:
            numero1 = row[0]
            numero2 = row[1]

        try:
            comando = f"select * from arquivadosxml where (numero='{numero1}' or numero='{numero2}') and despacho in ('21.1','21.2','21.6','21.7') and anulado=0"
            cursor.execute(comando)
            resultado2 = cursor.fetchall()
            if resultado2 != None:
                for row in resultado2:
                    data = row[3]
                    comando = f"update arqpatentes set extincao='{data}' where numero='{numero}'"
                    cursor.execute(comando)
                    conexao.commit()
                    print(comando+";")
                    total = total + 1
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Conferir user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Banco de dados inativo")
            else:
                print(err)
                print("Erro "+str(numero)+" na gravação na tabela arqpatentes")
                
        #if (total==2000): break

    print("Atualiza campo extincao na tabela arqpatentes verificando as extincoes anuladas")
    comando = f"select * from arqpatentes where extincao is not null"
    total = 0
    cursor.execute(comando)
    resultado = cursor.fetchall()
    for row in resultado:
        numero = row[0]
        data = row[6]

        numero1 = numero
        numero2 = numero
        comando = f"select * from pimupi where numero1='{numero}' or numero2='{numero}'"
        cursor.execute(comando)
        resultado2 = cursor.fetchall()
        for row in resultado2:
            numero1 = row[0]
            numero2 = row[1]

        try:
            comando = f"select * from arquivadosxml where (numero='{numero1}' or numero='{numero2}') and data='{data}' and despacho in ('21.1','21.2','21.6','21.7') and anulado<>0"
            cursor.execute(comando)
            resultado2 = cursor.fetchone()
            if resultado2 != None:
                comando = f"update arqpatentes set extincao=null where numero='{numero}'"
                cursor.execute(comando)
                conexao.commit()
                print(comando+";")
                total = total + 1
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Conferir user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Banco de dados inativo")
            else:
                print(err)
                print("Erro "+str(numero)+" na gravação na tabela arqpatentes")
                
        #if (total==2000): break

    print("Processamento encerrado")
    return

AtualizaExtincao()

Atualiza campo extincao na tabela arqpatentes
update arqpatentes set extincao='2022-12-20' where numero='102015005828';
update arqpatentes set extincao='2022-12-20' where numero='112012021401';
update arqpatentes set extincao='2022-12-20' where numero='112012021498';
update arqpatentes set extincao='2022-12-20' where numero='112012021532';
update arqpatentes set extincao='2022-12-20' where numero='112015020099';
update arqpatentes set extincao='2022-12-20' where numero='112015020405';
update arqpatentes set extincao='2022-12-20' where numero='112015020963';
update arqpatentes set extincao='2022-12-20' where numero='112016019841';
update arqpatentes set extincao='2022-12-20' where numero='112017003673';
update arqpatentes set extincao='2022-12-20' where numero='202017004089';
update arqpatentes set extincao='2022-12-20' where numero='PI0010926';
update arqpatentes set extincao='2022-12-20' where numero='PI0112138';
update arqpatentes set extincao='2022-12-20' where numero='PI0113727';
u

In [28]:
numero_tag = re.compile("<numero inid=\"21\">(.*?)</numero>") 
numero_a2_tag = re.compile("<numero inid=\"21\" kindcode=\"A2\">(.*?)</numero>") 
numero_e2_tag = re.compile("<numero inid=\"21\" kindcode=\"E2\">(.*?)</numero>") 
numero_u2_tag = re.compile("<numero inid=\"21\" kindcode=\"U2\">(.*?)</numero>") 
numero_a8_tag = re.compile("<numero inid=\"21\" kindcode=\"A8\">(.*?)</numero>") 
numero_u8_tag = re.compile("<numero inid=\"21\" kindcode=\"U8\">(.*?)</numero>") 
numero_21b1_tag = re.compile("<numero inid=\"21\" kindcode=\"B1\">(.*?)</numero>") 
numero_21b8_tag = re.compile("<numero inid=\"21\" kindcode=\"B8\">(.*?)</numero>") 
numero_21y1_tag = re.compile("<numero inid=\"21\" kindcode=\"Y1\">(.*?)</numero>") 
numero_21y8_tag = re.compile("<numero inid=\"21\" kindcode=\"Y8\">(.*?)</numero>") 
numero_21e8_tag = re.compile("<numero inid=\"21\" kindcode=\"E8\">(.*?)</numero>") 

numero_11a8_tag = re.compile("<numero inid=\"11\" kindcode=\"A8\">(.*?)</numero>") 
numero_b1_tag = re.compile("<numero inid=\"11\" kindcode=\"B1\">(.*?)</numero>") 
numero_b8_tag = re.compile("<numero inid=\"11\" kindcode=\"B8\">(.*?)</numero>") 
numero_y1_tag = re.compile("<numero inid=\"11\" kindcode=\"Y1\">(.*?)</numero>") 
numero_y8_tag = re.compile("<numero inid=\"11\" kindcode=\"Y8\">(.*?)</numero>") 
numero_c8_tag = re.compile("<numero inid=\"11\" kindcode=\"C8\">(.*?)</numero>") 
numero_f1_tag = re.compile("<numero inid=\"11\" kindcode=\"F1\">(.*?)</numero>") 
numero_f8_tag = re.compile("<numero inid=\"11\" kindcode=\"F8\">(.*?)</numero>") 
numero_g8_tag = re.compile("<numero inid=\"11\" kindcode=\"G8\">(.*?)</numero>") 

def DetectaNumero(numero,processo):
    if len(numero)==0:
        numero=numero_a2_tag.findall(processo)
        if len(numero)==0:
            numero=numero_u2_tag.findall(processo)
            if len(numero)==0:
                numero=numero_a8_tag.findall(processo)
                if len(numero)==0:
                    numero=numero_u8_tag.findall(processo)
                    if len(numero)==0:
                        numero=numero_b1_tag.findall(processo)
                        if len(numero)==0:
                            numero=numero_b8_tag.findall(processo)
                            if len(numero)==0:
                                numero=numero_y1_tag.findall(processo)
                                if len(numero)==0:
                                    numero=numero_y8_tag.findall(processo)
                                    if len(numero)==0:
                                        numero=numero_e2_tag.findall(processo)
                                        if len(numero)==0:
                                            numero=numero_21b1_tag.findall(processo)
                                            if len(numero)==0:
                                                numero=numero_21b8_tag.findall(processo)
                                                if len(numero)==0:
                                                    numero=numero_21y1_tag.findall(processo)
                                                    if len(numero)==0:
                                                        numero=numero_21y8_tag.findall(processo)
                                                        if len(numero)==0:
                                                            numero=numero_c8_tag.findall(processo)
                                                            if len(numero)==0:
                                                                numero=numero_f1_tag.findall(processo)
                                                                if len(numero)==0:
                                                                    numero=numero_11a8_tag.findall(processo)
                                                                    if len(numero)==0:
                                                                        numero=numero_21e8_tag.findall(processo)
                                                                        if len(numero)==0:
                                                                            numero=numero_f8_tag.findall(processo)
                                                                            if len(numero)==0:
                                                                                numero=numero_g8_tag.findall(processo)
    return numero

def BuscaDepositante(revistaInicio,revistaFim,numeroBuscado):
    nome_completo_buscado = None
    for revista in range(revistaInicio,revistaFim+1,1):
        filename, data_revista = LerFilename(revista)
        file_content = LerRPI(filename,data_revista)
        for line in file_content:
            processos=re.findall("<processo-patente>[\s\S]*</processo-patente>",line)
            for processo in processos:
                nome_completo = None
                numero=numero_tag.findall(processo)
                numero = DetectaNumero(numero,processo)
                numero = ', '.join(numero)  
                #titular_lista
                titulares_lista=re.findall("<titular-lista>[\s\S]*</titular-lista>",processo)
                for titular_lista in titulares_lista:
                    titulares=re.findall("<titular sequencia=\"1\" inid=\"71\">[\s\S]*</titular>",titular_lista)
                    for titular in titulares:
                        nome_completo=nome_completo_tag.findall(titular)
                        nome_completo = '; '.join(nome_completo)
                        enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                        for endereco in enderecos:
                            uf=re.findall("<uf>(.*?)</uf>",endereco)
                            uf = '; '.join(uf)
                            paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                            for pais in paises:
                                sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                                sigla = '; '.join(sigla)

                    titulares=re.findall("<titular sequencia=\"1\" inid=\"73\">[\s\S]*</titular>",titular_lista)
                    for titular in titulares:
                        nome_completo=nome_completo_tag.findall(titular)
                        nome_completo = '; '.join(nome_completo)
                        enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                        for endereco in enderecos:
                            uf=re.findall("<uf>(.*?)</uf>",endereco)
                            uf = '; '.join(uf)
                            paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                            for pais in paises:
                                sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                                sigla = '; '.join(sigla)

                numero=ConverteNumero(numero)
                if nome_completo is not None and numero==numeroBuscado:
                    nome_completo_buscado = nome_completo
                    nome_completo_buscado = nome_completo_buscado.replace("'", "\\'").strip()
                    #print(numero)
                    #print(nome_completo)
                    return nome_completo_buscado
                
    return nome_completo_buscado

comando = f"select * from arqpatentes where depositante is null"
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    numero = row[0]
    print(numero)
    depositante = BuscaDepositante(2661,2711,numero) # 2022 tem revistas de 2661 a 2712
    if depositante is not None:
        comando = f"update arqpatentes set depositante='{depositante}' where numero='{numero}'"
        cursor.execute(comando)
        conexao.commit()
        print(comando)

numero = '102021002341'
print(numero)
depositante = BuscaDepositante(2694,2694,numero) # 2022 tem revistas de 2661 a 2712
if depositante is not None:
    print(depositante)
    comando = f"update arqpatentes set depositante='{depositante}' where numero='{numero}'"
    cursor.execute(comando)
    conexao.commit()
    
print("Fim processamento")

102014006113
revistas/Patente_2661_04012022.xml
revistas/Patente_2662_11012022.xml
revistas/Patente_2663_18012022.xml
revistas/Patente_2664_25012022.xml
revistas/Patente_2665_01022022.xml
revistas/Patente_2666_08022022.xml
revistas/Patente_2667_15022022.xml
revistas/Patente_2668_22022022.xml
revistas/Patente_2669_03032022.xml
revistas/Patente_2670_08032022.xml
revistas/Patente_2671_15032022.xml
revistas/Patente_2672_22032022.xml
revistas/Patente_2673_29032022.xml
revistas/Patente_2674_05042022.xml
revistas/Patente_2675_12042022.xml
revistas/Patente_2676_19042022.xml
revistas/Patente_2677_26042022.xml
revistas/Patente_2678_03052022.xml
revistas/Patente_2679_10052022.xml
revistas/Patente_2680_17052022.xml
revistas/Patente_2681_24052022.xml
revistas/Patente_2682_31052022.xml
revistas/Patente_2683_07062022.xml
revistas/Patente_2684_14062022.xml
revistas/Patente_2685_21062022.xml
revistas/Patente_2686_28062022.xml
revistas/Patente_2687_05072022.xml
revistas/Patente_2688_12072022.xml
revista

revistas/Patente_2670_08032022.xml
revistas/Patente_2671_15032022.xml
revistas/Patente_2672_22032022.xml
revistas/Patente_2673_29032022.xml
revistas/Patente_2674_05042022.xml
revistas/Patente_2675_12042022.xml
revistas/Patente_2676_19042022.xml
revistas/Patente_2677_26042022.xml
revistas/Patente_2678_03052022.xml
revistas/Patente_2679_10052022.xml
revistas/Patente_2680_17052022.xml
revistas/Patente_2681_24052022.xml
revistas/Patente_2682_31052022.xml
revistas/Patente_2683_07062022.xml
revistas/Patente_2684_14062022.xml
revistas/Patente_2685_21062022.xml
revistas/Patente_2686_28062022.xml
revistas/Patente_2687_05072022.xml
revistas/Patente_2688_12072022.xml
revistas/Patente_2689_19072022.xml
revistas/Patente_2690_26072022.xml
revistas/Patente_2691_02082022.xml
revistas/Patente_2692_09082022.xml
revistas/Patente_2693_16082022.xml
revistas/Patente_2694_23082022.xml
revistas/Patente_2695_30082022.xml
revistas/Patente_2696_06092022.xml
revistas/Patente_2697_13092022.xml
revistas/Patente_269

revistas/Patente_2688_12072022.xml
revistas/Patente_2689_19072022.xml
revistas/Patente_2690_26072022.xml
revistas/Patente_2691_02082022.xml
revistas/Patente_2692_09082022.xml
revistas/Patente_2693_16082022.xml
revistas/Patente_2694_23082022.xml
revistas/Patente_2695_30082022.xml
revistas/Patente_2696_06092022.xml
revistas/Patente_2697_13092022.xml
revistas/Patente_2698_20092022.xml
revistas/Patente_2699_27092022.xml
revistas/Patente_2700_04102022.xml
revistas/Patente_2701_11102022.xml
revistas/Patente_2702_18102022.xml
revistas/Patente_2703_25102022.xml
revistas/Patente_2704_01112022.xml
revistas/Patente_2705_08112022.xml
revistas/Patente_2706_16112022.xml
revistas/Patente_2707_22112022.xml
revistas/Patente_2708_29112022.xml
revistas/Patente_2709_06122022.xml
revistas/Patente_2710_13122022.xml
revistas/Patente_2711_20122022.xml
update arqpatentes set depositante='CLAYTON CRISTIANO DIAS DE MATTOS' where numero='102021011010'
102021011015
revistas/Patente_2661_04012022.xml
revistas/Paten

revistas/Patente_2703_25102022.xml
revistas/Patente_2704_01112022.xml
revistas/Patente_2705_08112022.xml
revistas/Patente_2706_16112022.xml
revistas/Patente_2707_22112022.xml
revistas/Patente_2708_29112022.xml
revistas/Patente_2709_06122022.xml
revistas/Patente_2710_13122022.xml
revistas/Patente_2711_20122022.xml
update arqpatentes set depositante='COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA-COELBA; UNIVERSIDADE DE SÃO PAULO; OPTSENSYS INSTRUMENTAÇÃO ÓPTICA E ELETRÔNICA LTDA' where numero='102021011076'
102021011086
revistas/Patente_2661_04012022.xml
revistas/Patente_2662_11012022.xml
revistas/Patente_2663_18012022.xml
revistas/Patente_2664_25012022.xml
revistas/Patente_2665_01022022.xml
revistas/Patente_2666_08022022.xml
revistas/Patente_2667_15022022.xml
revistas/Patente_2668_22022022.xml
revistas/Patente_2669_03032022.xml
revistas/Patente_2670_08032022.xml
revistas/Patente_2671_15032022.xml
revistas/Patente_2672_22032022.xml
revistas/Patente_2673_29032022.xml
revistas/Patente_2674_

revistas/Patente_2663_18012022.xml
revistas/Patente_2664_25012022.xml
revistas/Patente_2665_01022022.xml
revistas/Patente_2666_08022022.xml
revistas/Patente_2667_15022022.xml
revistas/Patente_2668_22022022.xml
revistas/Patente_2669_03032022.xml
revistas/Patente_2670_08032022.xml
revistas/Patente_2671_15032022.xml
revistas/Patente_2672_22032022.xml
revistas/Patente_2673_29032022.xml
revistas/Patente_2674_05042022.xml
revistas/Patente_2675_12042022.xml
revistas/Patente_2676_19042022.xml
revistas/Patente_2677_26042022.xml
revistas/Patente_2678_03052022.xml
revistas/Patente_2679_10052022.xml
revistas/Patente_2680_17052022.xml
revistas/Patente_2681_24052022.xml
revistas/Patente_2682_31052022.xml
revistas/Patente_2683_07062022.xml
revistas/Patente_2684_14062022.xml
revistas/Patente_2685_21062022.xml
revistas/Patente_2686_28062022.xml
revistas/Patente_2687_05072022.xml
revistas/Patente_2688_12072022.xml
revistas/Patente_2689_19072022.xml
revistas/Patente_2690_26072022.xml
revistas/Patente_269

...

revistas/Patente_2685_21062022.xml
revistas/Patente_2686_28062022.xml
revistas/Patente_2687_05072022.xml
revistas/Patente_2688_12072022.xml
revistas/Patente_2689_19072022.xml
revistas/Patente_2690_26072022.xml
revistas/Patente_2691_02082022.xml
revistas/Patente_2692_09082022.xml
update arqpatentes set depositante='MITO INDUSTRIA METALURGICA LTDA' where numero='202022014748'
202022014795
revistas/Patente_2661_04012022.xml
revistas/Patente_2662_11012022.xml
revistas/Patente_2663_18012022.xml
revistas/Patente_2664_25012022.xml
revistas/Patente_2665_01022022.xml
revistas/Patente_2666_08022022.xml
revistas/Patente_2667_15022022.xml
revistas/Patente_2668_22022022.xml
revistas/Patente_2669_03032022.xml
revistas/Patente_2670_08032022.xml
revistas/Patente_2671_15032022.xml
revistas/Patente_2672_22032022.xml
revistas/Patente_2673_29032022.xml
revistas/Patente_2674_05042022.xml
revistas/Patente_2675_12042022.xml
revistas/Patente_2676_19042022.xml
revistas/Patente_2677_26042022.xml
revistas/Patent

revistas/Patente_2671_15032022.xml
revistas/Patente_2672_22032022.xml
revistas/Patente_2673_29032022.xml
revistas/Patente_2674_05042022.xml
revistas/Patente_2675_12042022.xml
revistas/Patente_2676_19042022.xml
revistas/Patente_2677_26042022.xml
revistas/Patente_2678_03052022.xml
revistas/Patente_2679_10052022.xml
revistas/Patente_2680_17052022.xml
revistas/Patente_2681_24052022.xml
revistas/Patente_2682_31052022.xml
revistas/Patente_2683_07062022.xml
revistas/Patente_2684_14062022.xml
revistas/Patente_2685_21062022.xml
revistas/Patente_2686_28062022.xml
revistas/Patente_2687_05072022.xml
revistas/Patente_2688_12072022.xml
revistas/Patente_2689_19072022.xml
revistas/Patente_2690_26072022.xml
revistas/Patente_2691_02082022.xml
revistas/Patente_2692_09082022.xml
revistas/Patente_2693_16082022.xml
revistas/Patente_2694_23082022.xml
revistas/Patente_2695_30082022.xml
update arqpatentes set depositante='MICHAEL HERBERT PEDRO' where numero='202022016340'
202022016822
revistas/Patente_2661_040

revistas/Patente_2671_15032022.xml
revistas/Patente_2672_22032022.xml
revistas/Patente_2673_29032022.xml
revistas/Patente_2674_05042022.xml
revistas/Patente_2675_12042022.xml
revistas/Patente_2676_19042022.xml
revistas/Patente_2677_26042022.xml
revistas/Patente_2678_03052022.xml
revistas/Patente_2679_10052022.xml
revistas/Patente_2680_17052022.xml
revistas/Patente_2681_24052022.xml
revistas/Patente_2682_31052022.xml
revistas/Patente_2683_07062022.xml
revistas/Patente_2684_14062022.xml
revistas/Patente_2685_21062022.xml
revistas/Patente_2686_28062022.xml
revistas/Patente_2687_05072022.xml
revistas/Patente_2688_12072022.xml
revistas/Patente_2689_19072022.xml
revistas/Patente_2690_26072022.xml
revistas/Patente_2691_02082022.xml
revistas/Patente_2692_09082022.xml
revistas/Patente_2693_16082022.xml
revistas/Patente_2694_23082022.xml
revistas/Patente_2695_30082022.xml
revistas/Patente_2696_06092022.xml
revistas/Patente_2697_13092022.xml
update arqpatentes set depositante='RUI MANUEL DIAS FER

revistas/Patente_2663_18012022.xml
revistas/Patente_2664_25012022.xml
revistas/Patente_2665_01022022.xml
revistas/Patente_2666_08022022.xml
revistas/Patente_2667_15022022.xml
revistas/Patente_2668_22022022.xml
revistas/Patente_2669_03032022.xml
revistas/Patente_2670_08032022.xml
revistas/Patente_2671_15032022.xml
revistas/Patente_2672_22032022.xml
revistas/Patente_2673_29032022.xml
revistas/Patente_2674_05042022.xml
revistas/Patente_2675_12042022.xml
revistas/Patente_2676_19042022.xml
revistas/Patente_2677_26042022.xml
revistas/Patente_2678_03052022.xml
revistas/Patente_2679_10052022.xml
revistas/Patente_2680_17052022.xml
revistas/Patente_2681_24052022.xml
revistas/Patente_2682_31052022.xml
revistas/Patente_2683_07062022.xml
revistas/Patente_2684_14062022.xml
revistas/Patente_2685_21062022.xml
revistas/Patente_2686_28062022.xml
revistas/Patente_2687_05072022.xml
revistas/Patente_2688_12072022.xml
revistas/Patente_2689_19072022.xml
revistas/Patente_2690_26072022.xml
revistas/Patente_269

revistas/Patente_2682_31052022.xml
revistas/Patente_2683_07062022.xml
revistas/Patente_2684_14062022.xml
revistas/Patente_2685_21062022.xml
revistas/Patente_2686_28062022.xml
revistas/Patente_2687_05072022.xml
revistas/Patente_2688_12072022.xml
revistas/Patente_2689_19072022.xml
revistas/Patente_2690_26072022.xml
revistas/Patente_2691_02082022.xml
revistas/Patente_2692_09082022.xml
revistas/Patente_2693_16082022.xml
revistas/Patente_2694_23082022.xml
revistas/Patente_2695_30082022.xml
revistas/Patente_2696_06092022.xml
revistas/Patente_2697_13092022.xml
revistas/Patente_2698_20092022.xml
revistas/Patente_2699_27092022.xml
revistas/Patente_2700_04102022.xml
revistas/Patente_2701_11102022.xml
revistas/Patente_2702_18102022.xml
update arqpatentes set depositante='MARCOS TAVARES PORSCH' where numero='202022020529'
202022020611
revistas/Patente_2661_04012022.xml
revistas/Patente_2662_11012022.xml
revistas/Patente_2663_18012022.xml
revistas/Patente_2664_25012022.xml
revistas/Patente_2665_010

In [89]:
def AtualizaDepositante():
    #atualiza campo depositante na tabela arqpatentes a partir da tabela publicados
    total = 0
    comando = f"select * from arqpatentes where depositante is null"
    cursor.execute(comando)
    resultado = cursor.fetchall()
    for row in resultado:
        numero = row[0]
        numero1 = numero
        numero2 = numero
        comando = f"select * from pimupi where numero1='{numero}' or numero2='{numero}'"
        cursor.execute(comando)
        resultado2 = cursor.fetchall()
        for row in resultado2:
            numero1 = row[0]
            numero2 = row[1]

        comando = f"select * from publicados where (numero='{numero1}' or numero='{numero2}')"
        cursor.execute(comando)
        resultado2 = cursor.fetchall()
        for row in resultado2:
            depositante = row[2]
            depositante = depositante.replace("'", "\\'").strip()
            if depositante is not None:
                comando = f"update arqpatentes set depositante='{depositante}' where numero='{numero}'"
                cursor.execute(comando)
                conexao.commit()
                print(comando+";")

        total = total + 1
        #if (total==200): break

    print("Processamento encerrado")
    return

AtualizaDepositante()

Processamento encerrado


In [17]:
df1 = data_frame.loc[data_frame['numero_new']=='112021020724']
df1.head(1)
len(df1)
#id = df1.index[0]
#id=0
#print(id)
#df1.loc[id]
#df1.loc[id].despacho

0

In [144]:
cursor = conexao.close()